In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/en-core-web-md/en_core_web_md/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/__init__.py
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/tokenizer
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSES_SOURCES
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSE
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/README.md
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/accuracy.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/config.cfg
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/attribute_ruler/patterns
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/vectors
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/key2row
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/vectors.cfg
/kaggle/input/en-core-we

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
import spacy

nlp = spacy.load("../input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

In [4]:
from string import punctuation as punc
from collections import defaultdict
import spacy
from string import punctuation as punc
from tqdm import tqdm

def process_text(text):
    doc = nlp(text)
    lemmas = []
    stops = 0
    pos = defaultdict(int,{'VERB': 0, "NOUN": 0})
    answer_length = len(text.split())

    for token in doc:
        if token.is_stop:
            stops += 1
        else:
            if token.pos_ == "VERB":
                pos[token.pos_] += 1
            elif token.pos_ == "NOUN":
                pos[token.pos_] += 1
            lemmas.append(token.lemma_.lower())
    pos = dict(pos)
    return ' '.join(lemmas), stops, pos['VERB'], pos['NOUN'], answer_length

def count_punc(text):
    return len([p for p in text if p in punc])

# Overlap Score
def overlap(sample):
    answer = set(sample['text'].lower().split())
    context = set(sample['prompt_text'].lower().split())
    intersection = len(context.intersection(answer))
    difference = len(context.difference(answer))
    union = len(context.union(answer))
    overlap_score = intersection/float(len(context))
    jaccard_similarity = intersection/union
    return intersection, difference, union, overlap_score, jaccard_similarity


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import TruncatedSVD,PCA

def preprocess_data(summary_df, prompt_df, ngram_range=(1,1)):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')
    train_df = train_df.assign(
    full_text='Title:\n' + train_df['prompt_title'] +
              '\nQuestion:\n' + train_df['prompt_question'] +
              '\nText:\n' + train_df['prompt_text']
    )

    print('processing text data')

    # Refactored text processing
    def process_texts(column_name):
        return zip(*train_df[column_name].apply(process_text).tolist())

    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())

    train_df['lemmas'],train_df['summary_stopwords'],train_df['summary_verbs'], train_df['summary_nouns'], train_df['length_of_summary'] = process_texts('text')
    train_df['full_text'], train_df['full_text_stopwords'],train_df['full_text_verbs'], train_df['full_text_nouns'], train_df['length_of_full_text'] = process_texts('full_text')

    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['length_of_summary', 'summary_punctuation','summary_stopwords','summary_verbs','summary_nouns',
            'intersection','difference','union', 'overlap_score', 'jaccard_similarity',
            'full_text_stopwords','full_text_verbs','full_text_nouns','length_of_full_text','content', 'wording']
    
    print('applying vectorizer.....')
    # Vectorization
    global vectorizer, pca
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    transformer = TfidfTransformer()
    transformed_df = transformer.fit_transform(vectorizer.fit_transform(train_df['full_text']))
    summary_transformed = transformer.fit_transform(vectorizer.transform(train_df['lemmas']))

    print('scaling features.....')
    # Scaling
    scaler = MinMaxScaler(feature_range=(0, 10))
    X_summary_scaled = scaler.fit_transform(transformed_df.toarray())
    summary_scaled = scaler.transform(summary_transformed.toarray())

    print('decomposing features.......')
    # PCA Decomposition
    components = round(transformed_df.shape[1] * (40/100))
    pca = TruncatedSVD(n_components=components,algorithm='arpack', random_state=42)
    pca_bag = pca.fit_transform(X_summary_scaled)
    summary_bag = pca.transform(summary_scaled)
    stacked_bag = np.hstack((pca_bag, summary_bag, train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

start_time = time.time()

stacked_bag, student_ids = preprocess_data(summary_df,prompt_df,ngram_range=(1,1))

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")

processing text data
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......
collecting outputs.........
Preprocessed function executed in: 1228.82 seconds


In [7]:
X_train,X_test,y_train,y_test = train_test_split(stacked_bag[:,:-2] ,stacked_bag[:,-2:],test_size=0.1,shuffle=True,random_state=11)

In [8]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

mcrmse_scorer = make_scorer(mcrmse, greater_is_better=False)
best_params = {'max_depth': 10,'max_features': 0.55,'min_samples_split': 10,'n_estimators': 300}
model_rf = MultiOutputRegressor(RandomForestRegressor(**best_params,n_jobs=-1, random_state=64),n_jobs=-1)

In [9]:
model_rf.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=10,
                                                     max_features=0.55,
                                                     min_samples_split=10,
                                                     n_estimators=300,
                                                     n_jobs=-1,
                                                     random_state=64),
                     n_jobs=-1)

In [10]:
y_preds = model_rf.predict(X_test)

In [11]:
mcrmse(y_test,y_preds)

0.5338831158761024

In [12]:
# model_rf.fit(stacked_bag[:,:-2], stacked_bag[:,-2:])

In [13]:
def preprocess_data(summary_df, prompt_df, vectorizer=vectorizer, pca=pca):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')
    train_df = train_df.assign(
    full_text='Title:\n' + train_df['prompt_title'] +
              '\nQuestion:\n' + train_df['prompt_question'] +
              '\nText:\n' + train_df['prompt_text']
    )

    print('processing text data')

    # Refactored text processing
    def process_texts(column_name):
        return zip(*train_df[column_name].apply(process_text).tolist())

    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())

    train_df['lemmas'],train_df['summary_stopwords'],train_df['summary_verbs'], train_df['summary_nouns'], train_df['length_of_summary'] = process_texts('text')
    train_df['full_text'], train_df['full_text_stopwords'],train_df['full_text_verbs'], train_df['full_text_nouns'], train_df['length_of_full_text'] = process_texts('full_text')

    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['length_of_summary', 'summary_punctuation','summary_stopwords','summary_verbs',
            'summary_nouns','intersection','difference','union', 'overlap_score', 
            'jaccard_similarity','full_text_stopwords','full_text_verbs',
            'full_text_nouns','length_of_full_text']
    
    print('applying vectorizer.....')
    # Vectorization
    transformer = TfidfTransformer()
    transformed_df = transformer.fit_transform(vectorizer.transform(train_df['full_text']))
    summary_transformed = transformer.fit_transform(vectorizer.transform(train_df['lemmas']))

    print('scaling features.....')
    # Scaling
    scaler = MinMaxScaler(feature_range=(0, 10))
    X_summary_scaled = scaler.fit_transform(transformed_df.toarray())
    summary_scaled = scaler.fit_transform(summary_transformed.toarray())

    print('decomposing features.......')
    # PCA Decomposition
    pca_bag = pca.transform(X_summary_scaled)
    summary_bag = pca.transform(summary_scaled)
    stacked_bag = np.hstack((pca_bag, summary_bag, train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [14]:
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

In [15]:
test_df,ids = preprocess_data(test_summary, test_prompts)

processing text data
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......
collecting outputs.........


In [16]:
y_preds = model_rf.predict(test_df)

In [17]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=ids).reset_index()
df_test = df_test.rename({'index':'student_id'},axis=1)

In [18]:
df_test

,student_id,content,wording
0,000000ffffff,-1.291684,-1.167582
1,222222cccccc,-1.291684,-1.167582
2,111111eeeeee,-1.291684,-1.167582
3,333333dddddd,-1.291684,-1.167582


In [19]:
df_test.to_csv("submission.csv", index=False)